In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import random

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

## Research Questions

### 1) Is there a significant difference in the proportion of minorities within a given flaring buffer zone versus outside the buffer? 

#### Ultimate goal is a comparison of three different buffers (400, 1000, 2000) following Czolowski et al, 2017 (they did 100; 400; 800; 1,000; 1,600; and 2,000 m)

#### Two comparisons: In_buffer versus all of California, and a geographically narrower version: in_buffer versus only the subset of the population in BGs that intersect a flare or are adjacent to a BG that intersects a flare. 

Null: Proportion of BG in a buffer is independent of minority status.  
Alt:  Proportion of BG in a buffer is related to minority status.
  
Test stat: Difference in weighted minority proportions.

Weighted minority proportion = sumproduct(minority_prop * intersect_prop) / sum(intersect_prop)
minority_prop is the proportion of minorities in a block group
intersect_prop is the proportion of BG area residing in the buffer zone

Permutation testing: Permute the intersect_prop variable, holding the fraction of minorities in each BG constant.
 
Process   
a) Calculate the actual minority proportions of the aggregrate in-buffer and out-buffer areas
Using 2km buffer for starters (Czolowski, 2017)  
b) Scramble the intersect_prop variable, holding the minority_prop variable constant.   
c) Calculate number of simulated proportions that match or exceed the actual proportion. Calculate p-value.  


## Load data, set project CRS

In [3]:
ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned

In [4]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")
print(f'Flares found: {len(ca_flares)}')

# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp", dtype={'bg_id': 'int', 'index':'int'})

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'shape_area_new'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

Flares found: 117
BGs found: 25607


In [5]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_state = ca_state.to_crs(meters_crs)
ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)

#### Discuss w RMI: dropping the three aquatic buffers

In [6]:
# There are three BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# IDs: 60839900000, 61119901000, 60379902000
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

### Break things back out of functions, troubleshoot step by step to find issue with the out_buffer calculation coming out to ~0.60 for all buffer sizes. 

In [7]:
ca_counties.rename(columns={'NAMELSAD':'CNTY_NAME'}, inplace=True)  # old:new. Match col names for merging

# subset to only cols needed
ca_bg_joined_sub = ca_bg_joined[['bg_id', 'CNTY_NAME', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'shape_area_new', 'geometry']].copy()

ca_bg_joined_sub.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

counties_sub = ca_counties[['CNTY_NAME', 'geometry']].copy()

counties_sub.rename(columns={'geometry':'cnty_geom'}, inplace=True)  # old:new. Match col names for merging


In [8]:
na_counts = ca_bg_joined_sub.isna().sum()


In [9]:
na_counts

bg_id              0
CNTY_NAME          0
ACSTOTPOP          0
MINORPOP           0
D_PM25_2          14
shape_area_new     0
bg_geom            0
dtype: int64

In [10]:
# add county names to social dataframe

ca_bg_joined_sub = pd.merge(ca_bg_joined_sub, counties_sub, 
                             on='CNTY_NAME', how='left')



In [11]:
type(ca_bg_joined_sub)

geopandas.geodataframe.GeoDataFrame

In [14]:
buffer_sizes = [2000]#, 400, 800, 1000, 1600, 2000]#, 50000, 100000]

for buffer_size in buffer_sizes:
    ca_flares, intersect_df = create_buffer_intersection(ca_flares, ca_bg_joined_sub, buffer_size)

#### Store for later use if necessary: subsetting to only intersected counties

In [ ]:
# # subset full social df down to just the rows that are in the intersected counties
# county_names = intersect_df['CNTY_NAME'].unique()
# social_counties = ca_bg_joined_sub[ca_bg_joined_sub['CNTY_NAME'].isin(county_names)]

#### Subset full social data down to just intersected BGs and BGs adjacent ("touching") to the intersected BGs. 

In [15]:
intersect_df = intersect_df.set_geometry('intersect_geom')
ca_bg_joined_sub = ca_bg_joined_sub.set_geometry('bg_geom')

intersect_mask = intersect_df['bg_id'].unique()
intersect_bgs = ca_bg_joined_sub[ca_bg_joined_sub['bg_id'].isin(intersect_mask)]

In [16]:
type(intersect_bgs)

geopandas.geodataframe.GeoDataFrame

In [17]:
len(intersect_bgs)

238

### Use the intersected BGs to subset the full df to only the BGs adjacent to the intersected BGs. 

In [18]:
inter_adj_df = gp.sjoin(ca_bg_joined_sub, intersect_bgs, how = "inner", predicate = 'intersects')

In [19]:
na_counts = inter_adj_df.isna().sum()


In [20]:
na_counts

bg_id_left              0
CNTY_NAME_left          0
ACSTOTPOP_left          0
MINORPOP_left           0
D_PM25_2_left           2
shape_area_new_left     0
bg_geom                 0
cnty_geom_left          0
index_right             0
bg_id_right             0
CNTY_NAME_right         0
ACSTOTPOP_right         0
MINORPOP_right          0
D_PM25_2_right          0
shape_area_new_right    0
cnty_geom_right         0
dtype: int64

In [21]:
# clean up df after sjoin brought a bunch of extra cols in
to_drop = []

for col in inter_adj_df.columns:
    
    # Check if the column name ends with '_left'
    if col.endswith('_left'):
        new_col = col[:-5]
        inter_adj_df = inter_adj_df.rename(columns={col:new_col})

    # Check if the column name ends with '_right'
    elif col.endswith('_right'):
        to_drop.append(col)

inter_adj_df = inter_adj_df.drop(to_drop, axis=1)


In [22]:
# ensure that the intersections, not the counties, are set as the geometry for calculations.
intersect_df = intersect_df.set_geometry('intersect_geom')  # set to the buffers rather than the points

# Create new 'area' column for the areas of the intersections
intersect_df['intersect_area'] = intersect_df.area

# Calculate the proportion of each block group within the buffer zone
intersect_df['intersect_prop'] = intersect_df['intersect_area'] / intersect_df['shape_area_new']



# merge the intersections data back into the (subsetted) social df. This creates a df of all BGs — those
# that contain intersections and those that do not — in the counties that have an intersection. 

# I'm creating a subset of California, in effect, to combat against the extremely skewed dataset when I 
# compare the in/out buffer proportions across all of Cali. 
bgs_intersections = pd.merge(inter_adj_df, 
                             intersect_df[['bg_id','intersect_geom','intersect_area', 'intersect_prop']], 
                             on=['bg_id'], 
                             how='left').fillna({'intersect_area': 0, 'intersect_prop': 0})

In [23]:
len(bgs_intersections)

2097

In [24]:
# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    bgs_intersections[var + '_intersect_count'] = bgs_intersections[var] * bgs_intersections['intersect_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    bgs_intersections[var + '_bg_totprop'] = np.where(bgs_intersections['ACSTOTPOP'] != 0, 
                                                      bgs_intersections[var] / bgs_intersections['ACSTOTPOP'], 0)

bgs_intersections['outintersect_prop'] = 1-bgs_intersections['intersect_prop']

# Calculate the in_buffer and out_buffer proportions for the sample
in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])


In [25]:
print(f" buffer: *{buffer_size}m*")
#print(f" mean intersection proportion*1000: {mean_intersect_prop:.10f}")
#print(f" mean out-intersection proportion: {mean_outintersect_prop:.10f}")
#print(f" mean intersection MINORPOP count: {mean_intersect_MINORcount:.10f}")
print(f" BGs in buffer: {len(intersect_df)} out of total: {len(bgs_intersections)}")
print(f" People in buffer: {round(sum(bgs_intersections['ACSTOTPOP_intersect_count'])):,} out of total: {sum(bgs_intersections['ACSTOTPOP']):,}")
print()
print(f" in_buffer prop: {in_buffer}")
print(f" out_buffer prop: {out_buffer}")
print()
# print(f" sampled in_buffer prop: {mean_in_buffer}")
# print(f" sampled out_buffer prop: {mean_out_buffer}")
# print()
# print("in buffer sample props")
# print(in_buffer_list)
# print()
# print("out buffer sample props")
# print(out_buffer_list)

# drop the geometries before writing to files; they're not needed for permutation
bgs_intersections_drop = bgs_intersections.drop(["bg_geom", "intersect_geom", "cnty_geom"], axis=1)    
filename = f"data/df_{buffer_size}m_forpermutation.tsv"
bgs_intersections_drop.to_csv(filename, sep="\t", index=False)


 buffer: *2000m*
 BGs in buffer: 238 out of total: 2097
 People in buffer: 981,583 out of total: 3,298,452

 in_buffer prop: 0.7497231480265352
 out_buffer prop: 0.6970300871560429



# Functions

In [43]:
def create_buffer_intersection(flares_df, social_df, buffer_size):
    #social_df = social_df[['bg_id', 'CNTY_NAME', 'ACSTOTPOP', 'MINORPOP',
    #                       'D_PM25_2', 'shape_area_new', 'bg_geom', 'cnty_geom']]
    
    social_df = social_df.set_geometry('bg_geom')
    
    #flares_df = set_geometry_buffer(flares_df, buffer_size)
    buffer_col = f"buffer_{buffer_size}m"
    flares_df[buffer_col] = flares_df['geometry'].buffer(distance=buffer_size)

    flares_df = flares_df.set_geometry(buffer_col)
    
    temp = flares_df.unary_union
    all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=flares_df.crs)  # convert back to geodf for processing
    
    intersect_temp = gp.overlay(social_df, all_buffers, how='intersection')  # could look at keep_geom=False
    
    intersect_temp.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging
    
    return flares_df, intersect_temp

In [119]:
def calculate_stats_adjacent(intersect_portions, social_df, buffer_size):

    intersect_portions_temp = intersect_portions
    intersect_mask = intersect_portions_temp['bg_id'].unique()

    # All intersected block groups
    intersect_bgs = social_df[social_df['bg_id'].isin(intersect_mask)]
    intersect_bgs = intersect_bgs.set_geometry('bg_geom')  # set to the buffers rather than the points

    # create new df of all intersected BGs plus adjacent BGs to those intersected
    inter_adj_df = gp.sjoin(social_df, intersect_bgs, how = "inner", predicate = 'intersects')
    inter_adj_df = inter_adj_df.loc[:, ~inter_adj_df.columns.str.endswith('_right')]
    inter_adj_df.columns = inter_adj_df.columns.str.replace('_left', '')

    # ensure that the intersections, not the counties, are set as the geometry for calculations.
    intersect_portions_temp = intersect_portions_temp.set_geometry('intersect_geom')  # set to the buffers rather than the points

    # Create new 'area' column for the areas of the intersections
    intersect_portions_temp['intersect_area'] = intersect_portions_temp.area

    # Calculate the proportion of each block group within the buffer zone
    intersect_portions_temp['intersect_prop'] = intersect_portions_temp['intersect_area'] / intersect_portions_temp['shape_area_new']


    # merge the intersections data back into the (subsetted) social df. This creates a df of all BGs — those
    # that contain intersections and those that do not — in the bgs that have an intersection plus BGs adjacent
    # to the intersected BGs. 

    # I'm creating a subset of California, in effect, to combat against the extremely skewed dataset when I 
    # compare the in/out buffer proportions across all of Cali. 

    bgs_intersections = pd.merge(inter_adj_df, 
                                 intersect_portions_temp[['bg_id','intersect_geom','intersect_area', 'intersect_prop']], 
                                 on=['bg_id'], 
                                 how='left').fillna({'intersect_area': 0, 'intersect_prop': 0})

    # Apply the proportion to each demographic variable to find counts by variable
    demo_vars = ['ACSTOTPOP', 'MINORPOP']
    for var in demo_vars:
        bgs_intersections[var + '_intersect_count'] = bgs_intersections[var] * bgs_intersections['intersect_prop']

    # find overall proportions for each demo var by dividing var count by respective total population    
    for var in demo_vars:
        bgs_intersections[var + '_bg_totprop'] = np.where(bgs_intersections['ACSTOTPOP'] != 0, 
                                                          bgs_intersections[var] / bgs_intersections['ACSTOTPOP'], 0)

    bgs_intersections['outintersect_prop'] = 1-bgs_intersections['intersect_prop']

    # Calculate the in_buffer and out_buffer proportions for the sample
    in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
    out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])

    print(f" buffer: *{buffer_size}m*")
    print(f" BGs in buffer: {len(intersect_portions_temp)} out of total: {len(bgs_intersections)}")
    print(f" People in buffer: {round(sum(bgs_intersections['ACSTOTPOP_intersect_count'])):,} out of total: {sum(bgs_intersections['ACSTOTPOP']):,}")
    print()
    print(f" in_buffer prop: {in_buffer}")
    print(f" out_buffer prop: {out_buffer}")
    print()

    #    drop the geometries before writing to files; they're not needed for permutation
    bgs_intersections_drop = bgs_intersections.drop(["bg_geom", "intersect_geom", "cnty_geom"], axis=1)    
    filename = f"data/adjacent_df_{buffer_size}m_forpermutation.tsv"
    bgs_intersections_drop.to_csv(filename, sep="\t", index=False)
    
    return


In [120]:
buffer_sizes = [400, 1000, 2000, 5000, 10000]#, 400, 800, 1000, 1600, 2000]#, 50000, 100000]

for buffer_size in buffer_sizes:
    ca_flares, intersect_portions = create_buffer_intersection(ca_flares, ca_bg_joined_sub, buffer_size)
    calculate_stats_adjacent(intersect_portions, ca_bg_joined_sub, buffer_size)



 buffer: *400m*
 BGs in buffer: 52 out of total: 583
 People in buffer: 6,781 out of total: 965,645

 in_buffer prop: 0.44450631636764576
 out_buffer prop: 0.6451717783510734

 buffer: *1000m*
 BGs in buffer: 90 out of total: 923
 People in buffer: 86,096 out of total: 1,484,558

 in_buffer prop: 0.5924038086079221
 out_buffer prop: 0.679858591011517

 buffer: *2000m*
 BGs in buffer: 238 out of total: 2097
 People in buffer: 981,583 out of total: 3,298,452

 in_buffer prop: 0.7497231480265352
 out_buffer prop: 0.6970300871560429

 buffer: *5000m*
 BGs in buffer: 1325 out of total: 10433
 People in buffer: 11,547,908 out of total: 16,605,683

 in_buffer prop: 0.7590792795236927
 out_buffer prop: 0.6749804191890185

 buffer: *10000m*
 BGs in buffer: 4124 out of total: 31498
 People in buffer: 43,488,216 out of total: 50,161,774

 in_buffer prop: 0.7244725191417997
 out_buffer prop: 0.6080469680979932



In [124]:
def calculate_stats_ca(intersect_portions, social_df, buffer_size):

    intersect_portions_temp = intersect_portions

    # ensure that the intersections, not the counties, are set as the geometry for calculations.
    intersect_portions_temp = intersect_portions_temp.set_geometry('intersect_geom')  # set to the buffers rather than the points

    # Create new 'area' column for the areas of the intersections
    intersect_portions_temp['intersect_area'] = intersect_portions_temp.area

    # Calculate the proportion of each block group within the buffer zone
    intersect_portions_temp['intersect_prop'] = intersect_portions_temp['intersect_area'] / intersect_portions_temp['shape_area_new']

    bgs_intersections = pd.merge(social_df, 
                                 intersect_portions_temp[['bg_id','intersect_geom','intersect_area', 'intersect_prop']], 
                                 on=['bg_id'], 
                                 how='left').fillna({'intersect_area': 0, 'intersect_prop': 0})

    # Apply the proportion to each demographic variable to find counts by variable
    demo_vars = ['ACSTOTPOP', 'MINORPOP']
    for var in demo_vars:
        bgs_intersections[var + '_intersect_count'] = bgs_intersections[var] * bgs_intersections['intersect_prop']

    # find overall proportions for each demo var by dividing var count by respective total population    
    for var in demo_vars:
        bgs_intersections[var + '_bg_totprop'] = np.where(bgs_intersections['ACSTOTPOP'] != 0, 
                                                          bgs_intersections[var] / bgs_intersections['ACSTOTPOP'], 0)

    bgs_intersections['outintersect_prop'] = 1-bgs_intersections['intersect_prop']

    # Calculate the in_buffer and out_buffer proportions for the sample
    in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
    out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])

    print(f" buffer: *{buffer_size}m*")
    print(f" BGs in buffer: {len(intersect_portions_temp)} out of total: {len(bgs_intersections)}")
    print(f" People in buffer: {round(sum(bgs_intersections['ACSTOTPOP_intersect_count'])):,} out of total: {sum(bgs_intersections['ACSTOTPOP']):,}")
    print()
    print(f" in_buffer prop: {in_buffer}")
    print(f" out_buffer prop: {out_buffer}")
    print()

    #    drop the geometries before writing to files; they're not needed for permutation
    bgs_intersections_drop = bgs_intersections.drop(["bg_geom", "intersect_geom", "cnty_geom"], axis=1)    
    filename = f"data/ca_df_{buffer_size}m_forpermutation.tsv"
    bgs_intersections_drop.to_csv(filename, sep="\t", index=False)
    
    return


In [123]:
buffer_sizes = [400, 1000, 2000]#, 5000, 10000]#, 400, 800, 1000, 1600, 2000]#, 50000, 100000]

for buffer_size in buffer_sizes:
    ca_flares, intersect_portions = create_buffer_intersection(ca_flares, ca_bg_joined_sub, buffer_size)
    calculate_stats_ca(intersect_portions, ca_bg_joined_sub, buffer_size)


 buffer: *400m*
 BGs in buffer: 52 out of total: 25604
 People in buffer: 2,602 out of total: 39,346,023

 in_buffer prop: 0.4445789157849305
 out_buffer prop: 0.6011868121852714

 buffer: *1000m*
 BGs in buffer: 90 out of total: 25604
 People in buffer: 22,852 out of total: 39,346,023

 in_buffer prop: 0.5936777332601038
 out_buffer prop: 0.601179569128563

 buffer: *2000m*
 BGs in buffer: 238 out of total: 25604
 People in buffer: 162,231 out of total: 39,346,023

 in_buffer prop: 0.7355340743107128
 out_buffer prop: 0.6006193006270928



## Analysis
### Functions

In [ ]:
def calc_proportions(df, buffer_size, gen_table):
    
    df.to_csv("data/testing_calc.csv")
    
    in_buffer = np.sum(df['MINORPOP_bg_totprop'] * df['intersect_prop']) / np.sum(df['intersect_prop'])
    out_buffer = (np.sum(df['MINORPOP_bg_totprop'] * (1 - df['intersect_prop'])) /
                  np.sum(1 - df['intersect_prop']))
    
    # table that compares the in-buffer proportions to the outside-buffer proportions
    temp = pd.DataFrame({'ID': [f"buffer_{buffer_size}"], 'Buffer': [buffer_size], 
                         'in_buffer': [in_buffer], 'out_buffer': [out_buffer]})
    
    
    gen_table = pd.concat([gen_table, temp], ignore_index=True)
    
    return gen_table
    

In [ ]:
def calc_counts(df, buffer_size, gen_table):

    in_buffer = df['MINORPOP_intersect_count'].sum()
    out_buffer = df['ACSTOTPOP'].sum() - in_buffer

    # table that compares the in-buffer proportions to the outside-buffer proportions
    temp = pd.DataFrame({'ID': [f"buffer_{buffer_size}"], 'Buffer': [buffer_size], 'in_buffer': [in_buffer], 'out_buffer': [out_buffer]})
    
    gen_table = pd.concat([gen_table, temp], ignore_index=True)
    
    return gen_table

In [ ]:
def combined_calcs(flares_df, social_df, ca_blockgroups, buffer_size, gen_table):
    
    intersect = create_buffer_intersection(flares_df, social_df, buffer_size)
    df = calc_intersect_stats(intersect, social_df, ca_blockgroups, buffer_size)
    
    gen_table = calc_proportions(df, buffer_size, gen_table)

    return gen_table

In [ ]:
def all_buffer_calcs(gen_table, buffer_sizes = [100, 400, 800, 1000, 1600, 2000]):

    for buffer_size in buffer_sizes:
        gen_table = combined_calcs(ca_flares, ca_bg_joined, ca_bg, buffer_size, gen_table.copy())  
        
    return gen_table

### Proportions table

In [ ]:
# initialize empty df
prop_table = pd.DataFrame(columns=['ID', "Buffer", 'in_buffer', 'out_buffer'])
name = prop_table.name = 'Proportions Table'

In [ ]:
prop_table = all_buffer_calcs(prop_table, buffer_sizes=[100, 400])
prop_table['Buffer'] = prop_table['Buffer'].astype(float)

In [ ]:
prop_table

#### *The out-buffer proportions are not being calculated correctly. Code seems right, though. Troubleshoot with JC*

In [ ]:
# scatterplot of buffer props

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(prop_table['Buffer'], prop_table['in_buffer'], label='Inside buffer')
ax.scatter(prop_table['Buffer'], prop_table['out_buffer'], label='Outside buffer')

# Set xticks to the buffer sizes
xticks = [100, 400, 800, 1000, 1600, 2000]
ax.set_xticks(xticks)

# Set xticklabels to the buffer sizes
xticklabels = [f"{b:,}" for b in xticks]
ax.set_xticklabels(xticklabels)
ax.set_xlabel('Buffer size (m)')
ax.set_ylabel('Proportion of MINORPOP')
ax.legend()
plt.show()


### Counts table

In [ ]:
# initialize empty df
count_table = pd.DataFrame(columns=['ID', "Buffer", 'in_buffer', 'out_buffer'])
name = count_table.name = 'Counts Table'

In [ ]:
# def combined_calcs(flares_df, social_df, ca_blockgroups, buffer_size, gen_table):
    
#     intersect = create_buffer_intersection(flares_df, social_df, buffer_size)
#     df = calc_intersect_stats(intersect, social_df, ca_blockgroups, buffer_size)
    
#     #gen_table = calc_proportions(df, buffer_size, gen_table)
#     gen_table = calc_counts(df, buffer_size, gen_table)

#     return gen_table

In [ ]:
count_table = all_buf_calcs(count_table)
count_table['Buffer'] = count_table['Buffer'].astype(float)

In [ ]:
count_table

In [ ]:
# scatterplot of buffer counts
# visualizing just in-buffer counts for readability. 

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(count_table['Buffer'], count_table['in_buffer'], label='Inside buffer')
#ax.scatter(count_table['Buffer'], count_table['out_buffer'], label='Outside buffer')

# Define the formatter function
def thousand_fmt(x, pos):
    return f"{x:,.0f}"
# Set the formatter function for the y-axis
ax.yaxis.set_major_formatter(mtick.FuncFormatter(thousand_fmt))

# Set xticks to the buffer sizes
xticks = [100, 400, 800, 1000, 1600, 2000]
ax.set_xticks(xticks)

# Set xticklabels to the buffer sizes
xticklabels = [f"{b:,}" for b in xticks]
ax.set_xticklabels(xticklabels)
ax.set_xlabel('Buffer size (m)')
ax.set_ylabel('Counts of MINORPOP')
ax.legend()
plt.show()

## Visualization with Folium

In [ ]:
# # Choropleth of Block Groups

# folium.Choropleth(intersect_df,
#                   data=intersect_df,
#                   columns = ['bg_id', 'D_PM25_2'], 
#                   key_on='feature.properties.bg_id',
#                   fill_color="Reds",
#                   fill_opacity=0.7,
#                   line_opacity=0.2,
#                   legend_name="PM2.5 Index",
#                  name="Block Groups").add_to(m)


In [ ]:
# Convert the GeoDataFrame to the same CRS as the folium map (if necessary)

# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)

In [ ]:
ca_bg_joined_sub.sample(1)

In [ ]:
# Feature Group: All CA block groups

def style_function2(feature):
    return {
        'fillColor': 'grey',
        'color': 'grey',
        'fillOpacity': 0.01
    }

all_ca_bg = folium.FeatureGroup(name='All CA BG', overlay=True)
folium.GeoJson(
    data=ca_bg_joined_sub['bg_geom'],
    style_function=style_function2
).add_to(all_ca_bg)
all_ca_bg.add_to(m)

In [ ]:
# Feature Group: Full BG geom for intersected Block Groups

def style_function4(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'fillOpacity': 0.05
    }

all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=intersect_bgs["bg_geom"],
    style_function=style_function4
).add_to(all_bg)
all_bg.add_to(m)

In [ ]:
# Feature Group: Intersect portion of intersected Block Groups

def style_function3(feature):
    return {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.05
    }

inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=intersect_df["intersect_geom"],
    style_function=style_function3
).add_to(inter_all_bg)
inter_all_bg.add_to(m)

In [ ]:
# Feature Group: Intersected and Adjacent Block Groups

def style_function5(feature):
    return {
        'fillColor': 'green',
        'color': 'green',
        'fillOpacity': 0.05
    }

inter_adj_fg = folium.FeatureGroup(name='Intersect & Adjacent BG', overlay=True)
folium.GeoJson(
    data=inter_adj_df["bg_geom"],
    style_function=style_function5
).add_to(inter_adj_fg)
inter_adj_fg.add_to(m)

In [ ]:
# # Feature Group: Buffers

# def style_function1(feature):
#     return {
#         'fillColor': 'red',
#         'color': 'red',
#         'fillOpacity': 0.1
#     }

# all_flares_buffers = folium.FeatureGroup(name='Flare Buffers', overlay=True)
# folium.GeoJson(
#     data=ca_flares["buffer_2000m"],
#     style_function=style_function1
# ).add_to(all_flares_buffers)
# all_flares_buffers.add_to(m)




In [ ]:
# # Feature Group: Counties

# # def style_function1(feature):
# #     return {
# #         'fillColor': 'red',
# #         'color': 'red',
# #         'fillOpacity': 0.2
# #     }

# # all_flares_buffers = folium.FeatureGroup(name='Flare Buffers', overlay=True)
# # folium.GeoJson(
# #     data=intersect_df["cnty_geom"],
# #     style_function=style_function1
# # ).add_to(all_flares_buffers)
# # all_flares_buffers.add_to(m)

In [ ]:
# # Add hover functionality
# test = test.to_crs("EPSG:3857")
# test = test.set_geometry('bg_geom')
# test_json = test.__geo_interface__

In [ ]:

# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = intersect_df,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['ID','CNTY_NAME'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)
# m

In [ ]:
# Flares
all_flares_points = folium.FeatureGroup(name='flare points', overlay=True)
folium.GeoJson(data=ca_flares["geometry"]).add_to(all_flares_points)
all_flares_points.add_to(m)

In [ ]:
m